<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at '<a href="#papermill-error-cell">In [2]</a>'.</span>

In [1]:
from eoxhub import check_compatibility
check_compatibility("user-2023.03-02", dependencies=[])

# Using the NASA VEDA EOAPI

This notebook is divided into two parts, each demonstrating the functionalities of the VEDA Earth Observation API (EOAPI).
1. Reading and visualizing one of the datasets from the VEDA data catalog
2. Using the EOAPI to generate a time-series of OMI (ozone monitoring instrument) NO₂ and SO₂ datasets

Author: Slesa Adhikari

#### 1. Reading and visualizing one of the datasets from the VEDA data catalog

**Import all the necesssary libraries**

Make sure you install these first using:

```bash
pip install pystac_client folium seaborn pandas
```

<span id="papermill-error-cell" style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">Execution using papermill encountered an exception here and stopped:</span>

In [2]:
# imports
import requests
from pystac_client import Client
import folium
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

ModuleNotFoundError: No module named 'pystac_client'

**Define the API endpoints**

The EOAPI is a combination of two components.
1. Data catalog - the [SpatioTemporal Asset Catalog (STAC)](https://stacspec.org/en/) specification is used to catalog the available datasets
2. Dynamic tile server - [TiTiler](https://developmentseed.org/titiler/) is used to dynamically serve cloud optimized geotiff (raster) data files

In [ ]:
STAC_API_URL = "https://staging-stac.delta-backend.com/"
RASTER_API_URL = "https://staging-raster.delta-backend.com"

Use the `pystac_client` library to interact with the STAC data catalog

In [ ]:
catalog = Client.open(STAC_API_URL)

List all the datasets (collections) in the catalog

In [ ]:
for collection in list(catalog.get_collections()):
    print(f"{collection.id} - {collection.title}")

Choose a collection to work with. We'll choose the `no2-monthly` dataset for this example.
This is the global nitrogen dioxide data organized into monthly metrics.

In [ ]:
collection_id = "no2-monthly"

Search all the items in the collection.

In [ ]:
search = catalog.search(collections=[collection_id])
items = list(search.items())
items

Load and inspect one of the item's assets. In the catalog, the cloud optimized geotiffs (COG) are indexed under `cog_default` asset.

In [ ]:
s3_uri = items[0].assets["cog_default"].href

Get statistics for the COG file using the TiTiler api endpoint `/cog/statistics`

In [ ]:
stats = requests.get(
    f"{RASTER_API_URL}/cog/statistics",
    params={"url": s3_uri}
).json()
stats

**Display the COG in a map**

Now, we'll make a request for imagery tiles to the raster API.

Define the query parameters for the request:
- collection
- item
- rescale
- colormap
- assets

In [ ]:
collection = collection_id
item = items[0].id
colormap = "rainbow"
assets = "cog_default"
rescale = f"{stats['1']['min']},{stats['1']['max']}"

Get the tileset json for the relevant item.

In [ ]:
tiles = requests.get(
    f"{RASTER_API_URL}/stac/tilejson.json?collection={collection}&item={item}&assets={assets}&colormap_name={colormap}&rescale={rescale}"
).json()

In [ ]:
tiles

Use the `/tiles` endpoint to visualize the file in a map

In [ ]:
m = folium.Map(
    zoom_start=10,
    scroll_wheel_zoom=True, 
    tiles=tiles["tiles"][0],
    attr="VEDA", 
    minzoom=0, 
    maxzoom=18,
)

In [ ]:
m

#### 2. Using the EOAPI to generate a time-series of OMI NO₂ and SO₂ datasets

There's a story published in the EODashboard with the following title:

> Air pollution in India, China and the U.S. have changed significantly over the past two decades

Link: https://eodashboard.org/story?id=air-pollution-us-india-china

The story talks about the trend of air pollution in India, China and the U.S. using the NO<sub>2</sub> and SO<sub>2</sub> readings taken from the OMI instrument

Here, we'll recreate the analysis using the EOAPI.

In [ ]:
# Here, we find the relevant collection ID for the dataset
collections = {
    "no2": "OMI_trno2-COG",
    "so2": "OMSO2PCA-COG",
}

Define the roughly similar Area of Interest (AOI) for each of the countries as seen in the story.

Disclaimer: Since the AOI is roughly similar, but not exactly the same, the results may differe slightly, but should maintain similar trends.

In [ ]:
aois = {
    "india": {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "coordinates": [
          [
            [
              84.44227371801799,
              25.276852788244952
            ],
            [
              81.73331688510166,
              25.379576397063317
            ],
            [
              81.40290450746915,
              20.640781701865322
            ],
            [
              84.09079123546121,
              20.59296261766137
            ],
            [
              84.44227371801799,
              25.276852788244952
            ]
          ]
        ],
        "type": "Polygon"
      }
    },
    "china": {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "coordinates": [
          [
            [
              118.14487188674968,
              40.38237805885373
            ],
            [
              112.59679754686567,
              40.39197699341523
            ],
            [
              112.78712023622006,
              32.015052150835814
            ],
            [
              117.937454307721,
              32.102440507249895
            ],
            [
              118.14487188674968,
              40.38237805885373
            ]
          ]
        ],
        "type": "Polygon"
      }
    },
    "usa": {
        "type": "Feature",
        "properties": {},
        "geometry": {
            "coordinates": [
            [
                [
                -80.16702521343733,
                41.73420113945659
                ],
                [
                -83.56446680395005,
                38.599369254919566
                ],
                [
                -82.00280661075571,
                37.54658260550103
                ],
                [
                -78.28140359718638,
                40.450899619800595
                ],
                [
                -80.16702521343733,
                41.73420113945659
                ]
            ]
            ],
            "type": "Polygon"
        }
    },
}

Define a function that takes the following params:
  1. `item`: a STAC item
  2. `geojson`: the geojson of the AOI

Using the `/cog/statistics/` endpoint of the raster API, we get back the statistics of the `item` (which corresponds to one COG file) within the given `geojson` AOI.

The statistics includes `min`, `max`, `mean`, `std`, etc.

In [ ]:
# the bounding box should be passed to the geojson param as a geojson Feature or FeatureCollection
def generate_stats(item, geojson):
    result = requests.post(
        f"{RASTER_API_URL}/cog/statistics", 
        params={
            "url": item.assets["cog_default"].href
        },
        json=geojson
    ).json()
    return {
        **result["properties"],
        "start_datetime": str(item.properties.get("datetime", item.properties.get("start_datetime")))[:4],
        "collection": item.collection_id
    }

Let's start out with the US 🇺🇸 !

We'll get all the items in the NO2 and SO2 collections and generate the statistics from them for the Ohio River Valley region of the United States.

In [ ]:
usa_aoi = aois["usa"]
items = list(catalog.search(collections=[collections["no2"], collections["so2"]]).items())
usa_stats = [
    generate_stats(item, usa_aoi)
    for item in items
]

Create a function that takes the statistics (which is a json) and converts it to a pandas dataframe in a format that'll make it easy to read and visualize.

We are only concerned with the `mean` statistics for this example. Specifically the change from the year 2005 as a percentage. We'll use pandas to calculate this change percentage and assign it to the `change` column.

In [ ]:
def clean_stats(stats_json) -> pd.DataFrame:
    # convert the stats_json as is to pandas dataframe
    df = pd.json_normalize(stats_json)
    # simple renaming for readability
    df.columns = [col.replace("statistics.1.", "") for col in df.columns]
    # create a date column from the start_datetime column
    df["date"] = pd.to_datetime(df["start_datetime"], format="mixed")
    # rename the collection to a more friendly name
    df["collection"] = df["collection"].map(lambda x: next(key for key, value in collections.items() if value == x))
    # sort the dataframe by the date column
    df = df.sort_values(by=["date"])
    # create a change column that calculates the change of the mean values from the value in 2005
    df["change"] = df.groupby("collection", group_keys=False)["mean"].apply(lambda x: x.div(x.iloc[0]).subtract(1).mul(100))
    return df

In [ ]:
cleaned_stats_df = clean_stats(usa_stats)

We'll now create a time-series of the change in mean values for NO2 and SO2 for the area in the US.

In [ ]:
plt.xticks([i for i in range(0, 21, 2)])
sns.set_style("darkgrid")
ax = sns.lineplot(
    x="start_datetime",
    y="change",
    hue="collection",
    data=cleaned_stats_df,
    palette=["#2196f3", "#ff5722"],
    style="collection",
    markers=["*", "d"],
)
ax.set_title("US - Ohio River Valley")
ax.set_xlabel("Years")
ax.set_ylabel("Change from 2005 (%)")
plt.legend(frameon=False, ncol=3)
plt.show()

Now, let's create a function that creates this trend graph, given the country.

In [ ]:
def create_chart(country):
    stats = [generate_stats(item, aois[country]) for item in list(catalog.search(collections=[collections["no2"], collections["so2"]]).items())]
    df = clean_stats(stats)

    # Create a chart using Seaborn
    plt.xticks([i for i in range(0, 21, 2)])
    sns.set_style("darkgrid")
    ax = sns.lineplot(
        x="start_datetime",
        y="change",
        hue="collection",
        data=df,
        palette=["#2196f3", "#ff5722"],
        style="collection",
        markers=["*", "d"],
        legend="full",
        # label=["no2", "so2"]
    )
    ax.set_title(country.title())
    ax.set_xlabel("Years")
    ax.set_ylabel("Change from 2005 (%)")
    ax.legend(["no2", "so2"])
    plt.legend(frameon=False, ncol=3)
    plt.show()

We can use this function to create charts for the rest of the AOIs.

India 🇮🇳

In [ ]:
create_chart("india")

China 🇨🇳

In [ ]:
create_chart("china")